# Knowledge Graph with You.com Search Integration

This notebook demonstrates how to use the temporal agents with knowledge graphs and You.com search API.

## Setup

Make sure you have:
1. Created a `.env` file with your API keys
2. Installed all dependencies with `uv sync`

In [ ]:
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / "src"))

from kg_graph_search.config import get_settings
from kg_graph_search.graph.database import KnowledgeGraphDB
from kg_graph_search.graph.models import Entity, Triplet, TemporalValidityRange
from kg_graph_search.search.you_client import YouAPIClient

# Load settings
settings = get_settings()
print("✓ Configuration loaded")

## 1. You.com Search Example

In [ ]:
# Initialize You.com client
you_client = YouAPIClient(api_key=settings.you_api_key)

# Perform a search
query = "temporal knowledge graphs"
results = you_client.search(query, num_results=5)

print(f"Search results for: {results.query}\n")
for i, result in enumerate(results.results, 1):
    print(f"{i}. {result.title}")
    print(f"   URL: {result.url}")
    print(f"   {result.snippet[:150]}...\n")

## 2. Knowledge Graph Database Example

In [ ]:
# Initialize the knowledge graph database
db = KnowledgeGraphDB(settings.db_path)

# Create some example entities
alice = Entity(name="Alice", entity_type="Person", description="Software engineer")
bob = Entity(name="Bob", entity_type="Person", description="Data scientist")
company = Entity(name="TechCorp", entity_type="Organization", description="Technology company")

# Add entities to database
alice_id = db.add_entity(alice)
bob_id = db.add_entity(bob)
company_id = db.add_entity(company)

print(f"Added entities:")
print(f"  Alice (ID: {alice_id})")
print(f"  Bob (ID: {bob_id})")
print(f"  TechCorp (ID: {company_id})")

In [ ]:
from datetime import datetime

# Create relationships (triplets)
triplet1 = Triplet(
    subject_id=alice_id,
    subject_name="Alice",
    predicate="works_at",
    object_id=company_id,
    object_name="TechCorp",
    temporal_validity=TemporalValidityRange(
        start_time=datetime(2023, 1, 1),
        is_current=True
    ),
    confidence=1.0,
    source="manual_entry"
)

triplet2 = Triplet(
    subject_id=bob_id,
    subject_name="Bob",
    predicate="works_at",
    object_id=company_id,
    object_name="TechCorp",
    temporal_validity=TemporalValidityRange(
        start_time=datetime(2022, 6, 1),
        is_current=True
    ),
    confidence=1.0,
    source="manual_entry"
)

triplet3 = Triplet(
    subject_id=alice_id,
    subject_name="Alice",
    predicate="collaborates_with",
    object_id=bob_id,
    object_name="Bob",
    confidence=0.9,
    source="manual_entry"
)

# Add triplets to database
db.add_triplet(triplet1)
db.add_triplet(triplet2)
db.add_triplet(triplet3)

print("✓ Added relationships to knowledge graph")

In [ ]:
# Query the knowledge graph
alice_relationships = db.get_triplets_for_entity(alice_id, as_subject=True)

print(f"\nAlice's relationships:")
for rel in alice_relationships:
    print(f"  {rel.subject_name} --[{rel.predicate}]--> {rel.object_name}")
    if rel.temporal_validity:
        print(f"    Valid from: {rel.temporal_validity.start_time}")
        print(f"    Is current: {rel.temporal_validity.is_current}")
    print(f"    Confidence: {rel.confidence}")

In [ ]:
# Search for entities
search_results = db.search_entities("Tech")

print("\nSearch results for 'Tech':")
for entity in search_results:
    print(f"  {entity.name} ({entity.entity_type}): {entity.description}")

## 3. Combining Search with Knowledge Graph

This example shows how to use You.com search results to populate a knowledge graph.

In [ ]:
# Search for a topic
topic_query = "OpenAI GPT-4"
search_results = you_client.search(topic_query, num_results=3)

# Create entities from search results
for result in search_results.results:
    # Create an entity for the source
    source_entity = Entity(
        name=result.title,
        entity_type="WebPage",
        description=result.snippet[:200]
    )
    source_id = db.add_entity(source_entity)
    
    # Create a topic entity
    topic_entity = Entity(
        name=topic_query,
        entity_type="Topic",
        description=f"Search topic: {topic_query}"
    )
    topic_id = db.add_entity(topic_entity)
    
    # Create a relationship
    triplet = Triplet(
        subject_id=source_id,
        subject_name=result.title,
        predicate="discusses",
        object_id=topic_id,
        object_name=topic_query,
        confidence=0.8,
        source=result.url
    )
    db.add_triplet(triplet)

print(f"✓ Added {len(search_results.results)} search results to knowledge graph")

In [ ]:
# Clean up
db.close()
you_client.close()
print("\n✓ Cleanup complete")